### 웹 애플리케이션이 Ajax 통신을 이용하는 경우

웹 브라우저가 직접 서버에 요청하게 되면 수신받은 응답결과를 브라우저가 직접 처리하려고 한다.

이 때문에 화면이 변경되는 일이 발생하게 된다. 

만약 개발자가 화면 변경 없이 서버와 통신하고자 한다면(아이디 중복확인 등) Ajax통신을 사용하여 백그라운드에서 통신할 수 있으며 브라우저의 화면은 변경되지 않는다

1. 우클릭시 프레임 소스보기가 없는가.
2. 페이지 소스보기했을 때 데이터가 없는가.
3. 크롬의 개발자 도구에서 Network 탭의 XHR 영역에서 페이지를
   이용할 때 마다 통신 기록이 남는가.
   
만약 위의 조건이 맞는다면 Ajax를 사용하고 있는 것이고 XHR에 나오는 통신 기록이 요청할 주소가 된다.

In [1]:
import requests
import bs4
import pandas
import time
import pickle

# 한번이라도 저장한적이 있는지..
chk = False

In [2]:
# 날짜를 입력받아 그 주의 월요일 부터 일요일까지의 날짜
# 다음 주의 날짜를 반환하는 함수
def getDate(date) :
    url = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={date}&category=kbo'
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # print(soup)
    # li 태그들을 가져온다.
    li_list = soup.select('ul.tab > li')
    # print(li_list)
    
    date_list = []
    
    # li 태그의 수 만큼 반복한다.
    for idx, li_tag in enumerate(li_list) :
        # 첫 번째 li 태그라면 그냥 넘어간다.
        if idx == 0 :
            continue
        # a 태그를 추출한다.
        a_tag = li_tag.select('a')[0]
        # print(a_tag)
        # onclick 속성의 값을 가져온다.
        onclick = a_tag.attrs['onclick']
        # print(onclick)
        # 날짜 값을 가져온다.
        date_value = onclick.split("'")[1]
        # print(date_value)
        date_list.append(date_value)
        
    # print(date_list[-1])
    # print(date_list[:-1])
    return date_list[:-1], date_list[-1]
    

In [3]:
# 해당 날짜의 프로야구 데이터를 반환하는 함수
def getKBOData(now) :
    url = f'https://sports.news.naver.com/schedule/scoreBoard.nhn?date={now}&category=kbo'
    
    # 오류 발생시를 대비해 요청 주소를 기록한다.
    with open ('KBOLog.txt', 'at') as fp :
        fp.write(url)
        fp.write('\n')
        
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    # 해당 날짜의 경기 데이터를 가지고 있는 ul 태그를 추출한다.    
    todaySchedule = soup.select('#todaySchedule')
    
    if len(todaySchedule) > 0 :
        # 내부의 모든 li 태그를 가져온다.
        li_list = todaySchedule[0].select('li')
        # li 태그 개수만큼 반복한다.
        for li_tag in li_list :
            # class 속성값을 가져온다.
            class_value = li_tag.attrs['class']
            # print(class_value)
            
            data_list = []
            
            if class_value[0] == 'end' :
                data_list.append('경기종료')
                data_list.append(now)
                # 팀 데이터를 추출한다.
                team_tag = li_tag.select('p.vs_team > strong')            
                data_list.append(team_tag[1].text.strip())
                data_list.append(team_tag[0].text.strip())
                # 점수 데이터를 가져온다.
                team_score = li_tag.select('strong.vs_num')
                data_list.append(team_score[1].text.strip())
                data_list.append(team_score[0].text.strip())
                # 투수 이름을 가져온다.
                team_pitcher = li_tag.select('span.game_info > a')
                if len(team_pitcher) > 0 :
                    data_list.append(team_pitcher[1].text.strip())
                    data_list.append(team_pitcher[0].text.strip())
                else :
                    data_list.append('모름')
                    data_list.append('모름')
                    
                
            elif class_value[0] == 'before_game' :
                data_list.append('경기전')
                data_list.append(now)
                # 팀 데이터를 추출한다.
                team_tag = li_tag.select('p.vs_team > strong')            
                data_list.append(team_tag[1].text.strip())
                data_list.append(team_tag[0].text.strip())
                # 점수 데이터를 가져온다.
                data_list.append('-1')
                data_list.append('-1')
                # 투수 이름을 가져온다.
                data_list.append('모름')
                data_list.append('모름')
                
            global chk
            if chk == False :
                chk = True
                head_list = ['경기상태', '날짜', '어웨이팀', '홈팀',
                             '어웨이점수', '홈점수', '어웨이투수',
                             '홈투수']
                
                
                df = pandas.DataFrame([data_list])
                df.columns = head_list
                df.to_csv('KBO.csv', index=False, encoding='utf-8-sig')
            else :
                
                df = pandas.DataFrame([data_list])
                df.to_csv('KBO.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

In [11]:
chk = False

current_date = '20190311'

# print(date_list)
# print(next_week)
# 해당 주의 날짜 수만큼 반복한다.
while True:
    date_list, next_week = getDate(current_date)
    for now in date_list :
        print(f'{now} 수집중')
        getKBOData(now)
    if len(next_week) == 0:
        break
    else :
        current_date = next_week
print('수집완료')

20190311 수집중
20190312 수집중
20190313 수집중
20190314 수집중
20190315 수집중
20190316 수집중
20190317 수집중
20190318 수집중
20190319 수집중
20190320 수집중
20190321 수집중
20190322 수집중
20190323 수집중
20190324 수집중
20190325 수집중
20190326 수집중
20190327 수집중
20190328 수집중
20190329 수집중
20190330 수집중
20190331 수집중
20190401 수집중
20190402 수집중
20190403 수집중
20190404 수집중
20190405 수집중
20190406 수집중
20190407 수집중
20190408 수집중
20190409 수집중
20190410 수집중
20190411 수집중
20190412 수집중
20190413 수집중
20190414 수집중
20190415 수집중
20190416 수집중
20190417 수집중
20190418 수집중
20190419 수집중
20190420 수집중
20190421 수집중
20190422 수집중
20190423 수집중
20190424 수집중
20190425 수집중
20190426 수집중
20190427 수집중
20190428 수집중
20190429 수집중
20190430 수집중
20190501 수집중
20190502 수집중
20190503 수집중
20190504 수집중
20190505 수집중
20190506 수집중
20190507 수집중
20190508 수집중
20190509 수집중
20190510 수집중
20190511 수집중
20190512 수집중
20190513 수집중
20190514 수집중
20190515 수집중
20190516 수집중
20190517 수집중
20190518 수집중
20190519 수집중
20190520 수집중
20190521 수집중
20190522 수집중
20190523 수집중
20190524 수집중
20190525 수집중
20190526 수집중